In [10]:
import json
from requests_html import AsyncHTMLSession
from utils import getStringBetweenTwoWords, extract_and_stringify_object , printJsonToFile,printSoupToHtml
from Product import ProductDetailDTO
#rewrite scrape_Jarir_full function with the object extraction 
async def scrape_Jarir_full(url):  
    session = AsyncHTMLSession()
    response = await session.get(url)
    soup=response.content.decode()
    printSoupToHtml(soup, "jarir.html")
    stringObj = getStringBetweenTwoWords(soup, 'original:', ',related').replace('!1', 'true').replace('!0', 'false')
    obj = json.loads(extract_and_stringify_object(stringObj))
    #load the json from the file jarir.json
    #obj = json.load(open("jarir.json"))    
    # Extracting the values from the specs string
    specs = getStringBetweenTwoWords(str(obj), 'homedelivery_enable', 'check_availability_status')
    specs_list = [spec.strip() for spec in specs.split(',') if ":" in spec]
    concatenated_specs = ', '.join(specs_list)

    printJsonToFile(obj, "jarir.json")
    name_Local = obj.get("name", "")
    name_global = obj.get("GTM_name", "")
    isAvailable = obj.get("check_availability_status", 0) == 1
    PriceAfterDiscount = obj.get("final_price_ex_tax", obj.get("price", None))
    rating = None  
    
    images = ["https://ak-asset.jarir.com/akeneo-prod/asset/"+image["image"] for image in obj.get("media_gallery", []) if image.get("type") == "image"]
    item_data = ProductDetailDTO(name_Global=name_global,name_Local=name_Local, price=PriceAfterDiscount,images=images,productlink1=url,description_Global=concatenated_specs,is_available=isAvailable,rating=rating)
    return item_data
#Test
url = 'https://www.jarir.com/sa-en/en-sa/samsung-qa55s90dauxsa-smart-tv-632865.html?country=eg'
x=await scrape_Jarir_full(url)
print("name_global:",x.name_Global)
print("name_local:",x.name_Local)
print("isAvailable:",x.is_available)    
print("price:",x.price)
print("rating:",x.rating)
print("description:",x.description_Global)
print("Description_Local:",x.description_Local)
print("images:",x.images)



name_global: Samsung 55" Smart TV, 4K OLED, OLED, Graphite Black, QA55S90DAUXSA
name_local: Samsung 55" Smart TV, 4K OLED, OLED, Graphite Black, QA55S90DAUXSA
isAvailable: True
price: 6433.913
rating: None
description: ': 1, 'acce': 'AV Cable/Sound Bar/Home Theater/Wireless/Screen Cleaning Kit/Cleaning Wipes', 'cofa': 'Black', 'core': 'NQ4 AI Gen2', 'cote': 'Bluetooth/WiFi/Wireless', 'diba': 'OLED', 'enco': 'Graphite Black', 'gest': '144 Hz', 'hdmi': '4 Port HDMI', 'hdrf': 'HDR+', 'mxrs': '3840 X 2160p', 'opsy': 'Tizen 2.0', 'poso': '100 - 240 Volts', 'ptyp': 'Smart TV', 'scsz': '55"', 'scty': '4K OLED', 'seri': 'Samsung Series 9', 'spin': '2.1CH (40W)', 'tdsu': 'n/a', 'touc': 'n/a'
Description_Local: 
images: ['https://ak-asset.jarir.com/akeneo-prod/asset/a/0/7/8/a07812e89c9b053151055c4f59b4464632bcadd6_632865.jpg', 'https://ak-asset.jarir.com/akeneo-prod/asset/4/4/6/7/4467c368e2c7b81b83c7893795da388d4dbacd57_632865_1.jpg', 'https://ak-asset.jarir.com/akeneo-prod/asset/d/1/e/d/d1ed86c

In [2]:
# from requests_html import AsyncHTMLSession
# from utils import getStringBetweenTwoWords
# from Product import ProductDetailDTO

# async def scrape_Jarir_full(url):  
#     session = AsyncHTMLSession()
#     response = await session.get(url)
#     script_target_object=getStringBetweenTwoWords(response.content.decode(), 'window.__INITIAL_STATE__= ','</script>')
#     strat_index=response.content.decode().index('window.__INITIAL_STATE__')
#     end_index=response.content.decode().index('</script>',strat_index)
#     script_target_object=response.content.decode()[strat_index:end_index].replace('window.__INITIAL_STATE__=','')
#     price=getStringBetweenTwoWords(script_target_object, 'final_price_ex_tax:',',')
#     Title=getStringBetweenTwoWords(script_target_object, 'GTM_name:',',')
#     images_string = getStringBetweenTwoWords(script_target_object, 'media_gallery','tsk').split("},{")
#     images=[]
#     for i in range(len(images_string)):
#         images.append("https://ak-asset.jarir.com/akeneo-prod/asset/"+getStringBetweenTwoWords(images_string[i], 'image:"','",lab'))
#     item_data = ProductDetailDTO(name_Global=Title, price=price,images=images,productlink1=url)
#     return item_data

# #Test
# url = 'https://www.jarir.com/sa-en/default-category/msi-clae-a1m-gaming-consoles-and-handheld-631416.html'
# x=await scrape_Jarir_full(url)
# print(x.__dict__)
